In [7]:
from typing import List
from pydantic import BaseModel, Field, field_validator
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.runnables import RunnableLambda


# 1. THE CONTRACT (Output Contracts for AI Systems)
# Schemas define required AI outputs and specify fields/types
class PatientSummary(BaseModel):
    patient_name: str = Field(description="Full name of the patient")
    age: int = Field(description="Age in years")
    diagnoses: List[str] = Field(description="List of medical conditions")

    # Automatic Error Detection: Catching malformed output instantly
    @field_validator("age")
    @classmethod
    def check_age(cls, v: int) -> int:
        if v < 0 or v > 120:
            # Errors caught before crashing the system
            raise ValueError("Age must be between 0 and 120")
        return v


# 2. SETUP
parser = PydanticOutputParser(pydantic_object=PatientSummary)
llm = ChatOpenAI(model="gpt-4o", temperature=0)  # Temp 0 for predictability

# The prompt creates a "contractual" expectation via format_instructions
prompt = ChatPromptTemplate.from_template(
    "Extract info from this note.\n{format_instructions}\nNote: {note}"
).partial(format_instructions=parser.get_format_instructions())


# 3. THE REPAIR LOGIC (Validation Loops)
# This feedback loop links generation and validation
def validate_and_repair(output):
    try:
        # Step: Valid data becomes a typed object
        return parser.parse(output.content)
    except Exception as e:
        print(f"\n [CONTRACT VIOLATION]: {e}")

        # Corrective prompts issued on validation failure
        # We explicitly tell the LLM to skip conversational 'noise'
        repair_msg = (
            f"Your last response broke the contract: {e}. "
            "Return ONLY the raw JSON. No conversational text, no markdown blocks, no 'Here is the JSON'."
        )

        # Self-correction: Model aligns more closely with schema over time
        repair_chain = llm | parser
        return repair_chain.invoke(repair_msg)


# 4. THE CHAIN (Natural Transformation Step)
# Truth becomes a structural outcome of the pipeline
chain = prompt | llm | RunnableLambda(validate_and_repair)

# 5. EXECUTION
# "Chaos Mode": Passing data that triggers the error detection
messy_note = "Subject is Benjamin Button. He claims to be -10 years old due to a temporal anomaly."
result = chain.invoke({"note": messy_note})

print("\n Final Validated Object (Executable Data):")
print(f"Name: {result.patient_name}")
print(f"Age: {result.age}")
print(f"Diagnoses: {result.diagnoses}")


 [CONTRACT VIOLATION]: Failed to parse PatientSummary from completion {"patient_name": "Benjamin Button", "age": -10, "diagnoses": ["temporal anomaly"]}. Got: 1 validation error for PatientSummary
age
  Value error, Age must be between 0 and 120 [type=value_error, input_value=-10, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error
For troubleshooting, visit: https://docs.langchain.com/oss/python/langchain/errors/OUTPUT_PARSING_FAILURE 

 Final Validated Object (Executable Data):
Name: Benjamin Button
Age: 10
Diagnoses: ['temporal anomaly']
